In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams.update({'font.family':'Gabriola', 'font.serif':['Gabriola']})
plt.rcParams.update({'pdf.fonttype': 'truetype', 'font.size': 18})

In [5]:
train = pd.read_csv('data/train.csv',index_col=0)
test = pd.read_csv('data/test.csv')

In [6]:
train.shape, test.shape

((20758, 17), (13840, 17))

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20758 entries, 0 to 20757
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          20758 non-null  object 
 1   Age                             20758 non-null  float64
 2   Height                          20758 non-null  float64
 3   Weight                          20758 non-null  float64
 4   family_history_with_overweight  20758 non-null  object 
 5   FAVC                            20758 non-null  object 
 6   FCVC                            20758 non-null  float64
 7   NCP                             20758 non-null  float64
 8   CAEC                            20758 non-null  object 
 9   SMOKE                           20758 non-null  object 
 10  CH2O                            20758 non-null  float64
 11  SCC                             20758 non-null  object 
 12  FAF                             

In [10]:
categorical = [var for var in train.columns if train[var].dtype == 'object' and var !='NObeyesdad']

print(f'There are {len(categorical)} categorical variables')
categorical

There are 8 categorical variables


['Gender',
 'family_history_with_overweight',
 'FAVC',
 'CAEC',
 'SMOKE',
 'SCC',
 'CALC',
 'MTRANS']

In [11]:
numerical = [var for var in train.columns if train[var].dtype != 'object' and var !='NObeyesdad']

print(f'There are {len(categorical)} categorical variables')
numerical

There are 8 categorical variables


['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

EDA - univariate analysis

Categorical variables

In [13]:
import plotly.express as px
fig = px.box(train, y="Age")
fig.show()

In [14]:
import plotly.express as px
df = px.data.tips()
fig = px.box(df, y="total_bill")
fig.show()